TP Modulo 3 - Curso Python para Finanzas

# Importamos los modulos a ser utilizados.

In [1]:
import requests, pandas as pd
import numpy as np
import tqdm
import datetime
from  keys import *
from db import BD_CONNECTION
from sqlalchemy import create_engine

# Listamos los Tickers con los cuales vamos a trabajar. (La base siempre va a ser USDT)

In [2]:
tickers = ['BTC', 'ETH', 'LTC', 'ETC', 'XRP', 'EOS', 'BCH', 'BSV', 'TRX']

# Creamos la tabla en la cual vamos a guardar los datos de OKEX.

In [4]:
engine = create_engine(BD_CONNECTION)

create_table = '''
CREATE TABLE IF NOT EXISTS `okex` (
  `id` int(11) NOT NULL AUTO_INCREMENT,
  `ticker` varchar(20) DEFAULT '',
  `time` timestamp NULL DEFAULT NULL,
  `open` double DEFAULT NULL,
  `high` double DEFAULT NULL,
  `low` double DEFAULT NULL,
  `close` double DEFAULT NULL,
  `volume` double DEFAULT NULL,
  PRIMARY KEY (`id`),
  UNIQUE KEY `idx_ticker_time` (`ticker`,`time`)
) ENGINE=InnoDB AUTO_INCREMENT=1 DEFAULT CHARSET=latin1;
'''
engine.execute(create_table)

# Creamos una funcion para bajar los datos de OKEX.

In [5]:
def historialOkex(ticker,end):
    '''ticker=btc, end=datetime'''
    url = f'https://okex.com/api/spot/v3/instruments/{ticker}-USDT/history/candles'
    start = end + datetime.timedelta(minutes=300 - 1)

    params = {
        'start': start.strftime('%Y-%m-%dT%H:%M:%S.000Z'),
        'end': end.strftime('%Y-%m-%dT%H:%M:%S.000Z'),
        'granularity': 60
    }
    print(params)
    r = requests.get(url, params=params)
    js = r.json()
    df = pd.DataFrame(js)

    df.columns = ['time', 'open', 'high', 'low', 'close', 'volume']

    df.time = pd.to_datetime(df.time)
    df.open = df.open.astype(float)
    df.high = df.high.astype(float)
    df.low = df.low.astype(float)
    df.close = df.close.astype(float)
    df.volume = df.volume.astype(float)

    df['ticker'] = ticker

    df.set_index('time', inplace=True)
    return df

# Iteramos y llamamos a la funcion creada

In [6]:
for ticker in tickers:
    print(ticker)


    finished = False
    while not finished:
        busquedaUltimaFecha = f'SELECT `id`,`time` FROM okex WHERE `ticker` = "{ticker}" ORDER BY `time` DESC limit 0,1'

        ultimaFecha = engine.execute(busquedaUltimaFecha).fetchone()        

        end = datetime.datetime.now() - datetime.timedelta(days=10)

        if (ultimaFecha):
            id = ultimaFecha[0]
            end = ultimaFecha[1]

            query_borrado = f'DELETE FROM okex WHERE `id`={id}'
            engine.execute(query_borrado)

        df=historialOkex(ticker=ticker,end=end)

        df.to_sql('okex', engine, if_exists='append')

        finished = len(df) < 2

        #print(df)

BTC
{'start': '2020-11-04T02:43:00.000Z', 'end': '2020-11-03T21:44:00.000Z', 'granularity': 60}
{'start': '2020-11-04T05:34:00.000Z', 'end': '2020-11-04T00:35:00.000Z', 'granularity': 60}
ETH
{'start': '2020-11-04T02:46:00.000Z', 'end': '2020-11-03T21:47:00.000Z', 'granularity': 60}
{'start': '2020-11-04T05:34:00.000Z', 'end': '2020-11-04T00:35:00.000Z', 'granularity': 60}
LTC
{'start': '2020-11-04T02:48:00.000Z', 'end': '2020-11-03T21:49:00.000Z', 'granularity': 60}
{'start': '2020-11-04T05:34:00.000Z', 'end': '2020-11-04T00:35:00.000Z', 'granularity': 60}
ETC
{'start': '2020-11-04T02:50:00.000Z', 'end': '2020-11-03T21:51:00.000Z', 'granularity': 60}
{'start': '2020-11-04T05:34:00.000Z', 'end': '2020-11-04T00:35:00.000Z', 'granularity': 60}
XRP
{'start': '2020-11-04T02:52:00.000Z', 'end': '2020-11-03T21:53:00.000Z', 'granularity': 60}
{'start': '2020-11-04T05:34:00.000Z', 'end': '2020-11-04T00:35:00.000Z', 'granularity': 60}
EOS
{'start': '2020-11-04T02:54:00.000Z', 'end': '2020-11-03

In [7]:
def fechaEnMs(dt):
    epoch = datetime.datetime.utcfromtimestamp(0)
    return int((dt - epoch).total_seconds() * 1000)

# Creamos la tabla en la cual vamos a guardar los datos de Binance.

In [8]:
create_table = '''
CREATE TABLE IF NOT EXISTS `binance` (
  `id` int(10) unsigned NOT NULL AUTO_INCREMENT,
  `ticker` varchar(20) DEFAULT '',
  `time` datetime DEFAULT NULL,
  `open` double DEFAULT NULL,
  `high` double DEFAULT NULL,
  `low` double DEFAULT NULL,
  `close` double DEFAULT NULL,
  `volume` double DEFAULT NULL,
  `open_time` bigint(20) DEFAULT NULL,
  `c_time` bigint(20) DEFAULT NULL,
  `q_volume` double DEFAULT NULL,
  `trades` bigint(20) DEFAULT NULL,
  `taker_base` double DEFAULT NULL,
  `taker_quote` double DEFAULT NULL,
  `ignore` bigint(20) DEFAULT NULL,
  PRIMARY KEY (`id`),
  KEY `idx_ticker_time` (`ticker`,`time`)
) ENGINE=InnoDB AUTO_INCREMENT=1 DEFAULT CHARSET=latin1;
'''
engine.execute(create_table)

# Creamos una funcion para bajar los datos de OKEX.

In [9]:
def DailySymbolVolSingleExchange(symbol, interval='1d', startTime=None, endTime=None, limit=30):
    '''bajadaSimple('BTCUSDT',interval='2h',startTime=1597719600000,endTime=1600398000000)'''
    url = 'https://api.binance.com/api/v3/klines'
    params = {'symbol':symbol, 'interval':interval,'startTime':startTime, 'endTime':endTime, 'limit':limit}
    r = requests.get(url, params=params)
    js = r.json()
    # Armo el dataframe
    cols = ['open_time','open','high','low','close','volume','c_time',
            'q_volume','trades','taker_base','taker_quote','ignore']
    df = pd.DataFrame(js, columns=cols)
    
    #Convierto los valores strings a numeros
    df = df.apply(pd.to_numeric)
    
    # Le mando indice de timestamp
    df['time'] = pd.to_datetime(df.open_time, unit='ms')
    #df.drop(['openTime','cTime','qVolume','trades','takerBase','takerQuote','Ignore'],axis=1,inplace=True)
    return df

# Iteramos y llamamos a la funcion creada

In [10]:
for ticker in tickers:

    print(ticker)

    finished = False
    while not finished:

        busquedaUltimaFecha = f'SELECT `id`,`time` FROM binance WHERE `ticker` = "{ticker}" ORDER BY `time` DESC limit 0,1'
        ultimaFecha = engine.execute(busquedaUltimaFecha).fetchone()
    
        start = datetime.datetime.now() - datetime.timedelta(days=10)
        
        if (ultimaFecha):
            id = ultimaFecha[0]
            start = ultimaFecha[1]

            query_borrado = f'DELETE FROM binance WHERE `id`={id}'
            engine.execute(query_borrado)
       
        print(start)
        
        start = fechaEnMs(start)

        df = DailySymbolVolSingleExchange(f'{ticker}USDT', interval='1m', startTime=start, limit=1000)

        df['ticker'] = ticker

        df.set_index('time', inplace=True)

        df.to_sql('binance', engine, if_exists='append')

        finished = len(df) < 2

BTC
2020-11-03 23:24:00
2020-11-04 00:37:00
ETH
2020-11-03 23:24:00
2020-11-04 00:37:00
LTC
2020-11-03 23:25:00
2020-11-04 00:37:00
ETC
2020-11-03 23:25:00
2020-11-04 00:37:00
XRP
2020-11-03 23:26:00
2020-11-04 00:37:00
EOS
2020-11-03 23:26:00
2020-11-04 00:37:00
BCH
2020-11-03 23:27:00
2020-11-04 00:37:00
BSV
2020-10-24 21:33:58.813474
TRX
2020-11-03 23:27:00
2020-11-04 00:37:00


# Creamos la vista

In [12]:
create_vista = '''
CREATE VIEW  margenes AS
SELECT b.ticker, b.time, b.close as 'binance_close', o.close as 'okex_close', ((b.close / o.close - 1.0) * 100.0) as 'diferencia'
FROM binance b JOIN okex o
ON b.ticker = o.ticker and b.time = o.time
where b.ticker != 'TRX'
order by abs(diferencia) desc
'''
engine.execute(create_vista)

# Leemos la Vista, en la cual vemos los margenes por minuto.

In [13]:
q='SELECT * FROM margenes'
pd.read_sql(q,engine)

,ticker,time,binance_close,okex_close,diferencia
0,ETC,2020-11-03 20:17:00,5.042,4.852,3.915911
1,BCH,2020-11-03 03:13:00,238.250,231.550,2.893544
2,BCH,2020-11-03 03:12:00,238.690,233.410,2.262114
3,BCH,2020-11-03 03:11:00,237.900,233.690,1.801532
4,BCH,2020-11-03 03:15:00,239.610,235.620,1.693405
...,...,...,...,...,...
102464,LTC,2020-11-03 23:36:00,53.980,53.980,0.000000
102465,LTC,2020-11-03 23:40:00,53.940,53.940,0.000000
102466,LTC,2020-11-03 23:48:00,53.880,53.880,0.000000
102467,LTC,2020-11-04 00:32:00,54.030,54.030,0.000000
